In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import re
import warnings
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Functions

In [2]:
def calculate_round_winner(cs2, max_round):
    for round_number in range(1, max_round + 1):
        conditions = []
        choices = ['Favorite', 'Underdog', 'Favorite', 'Underdog']

        # Define conditions based on round number
        if round_number == 1:
            conditions = [
                (cs2['Moneyline A'] < cs2['Moneyline B']) & (cs2['R1 Score (A)'] > cs2['R1 Score (B)']),
                (cs2['Moneyline A'] > cs2['Moneyline B']) & (cs2['R1 Score (A)'] > cs2['R1 Score (B)']),
                (cs2['Moneyline B'] < cs2['Moneyline A']) & (cs2['R1 Score (B)'] > cs2['R1 Score (A)']),
                (cs2['Moneyline B'] > cs2['Moneyline A']) & (cs2['R1 Score (B)'] > cs2['R1 Score (A)'])
            ]
        elif round_number == 2:
            conditions = [
                (cs2['Moneyline A'] < cs2['Moneyline B']) & (cs2['R2 Score (A)'] > cs2['R2 Score (B)']),
                (cs2['Moneyline A'] > cs2['Moneyline B']) & (cs2['R2 Score (A)'] > cs2['R2 Score (B)']),
                (cs2['Moneyline B'] < cs2['Moneyline A']) & (cs2['R2 Score (B)'] > cs2['R2 Score (A)']),
                (cs2['Moneyline B'] > cs2['Moneyline A']) & (cs2['R2 Score (B)'] > cs2['R2 Score (A)'])
            ]
        elif round_number == 3:
            # Check if the 'R3 Score' columns exist
            if 'R3 Score (A)' in cs2.columns and 'R3 Score (B)' in cs2.columns:
                conditions = [
                    (cs2['Moneyline A'] < cs2['Moneyline B']) & (cs2['R3 Score (A)'] > cs2['R3 Score (B)']),
                    (cs2['Moneyline A'] > cs2['Moneyline B']) & (cs2['R3 Score (A)'] > cs2['R3 Score (B)']),
                    (cs2['Moneyline B'] < cs2['Moneyline A']) & (cs2['R3 Score (B)'] > cs2['R3 Score (A)']),
                    (cs2['Moneyline B'] > cs2['Moneyline A']) & (cs2['R3 Score (B)'] > cs2['R3 Score (A)'])
                ]

        # Apply conditions and choices
        if conditions:
            cs2[f'Round {round_number} Winner'] = np.select(conditions, choices, default='')
        else:
            cs2[f'Round {round_number} Winner'] = ''

    return cs2

# CS Data

In [3]:
fp = '/content/drive/My Drive/Sports Betting/CS2 - Data.xlsx'

cs2 = pd.read_excel(fp)

In [4]:
cs2

,Date,Type,League,Best Of,Stars,Team A,Team B,Moneyline A,Moneyline B,R1 Score (A),...,R2 Score (A),R2 Score (B),R3 Score (A),R3 Score (B),R4 Score (A),R4 Score (B),R5 Score (A),R5 Score (B),Score Count (A),Score Count (B)
0,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,1,Aurora,Apeks,1.645,2.200,16,...,13,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Metizport,B8,1.833,1.909,13,...,13,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Monte,GamerLegion,1.741,2.050,12,...,13,11,13.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Ninjas in Pyjamas,BlesseD,1.364,3.000,13,...,13,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-03-31,Major,PGL CS2 Major Copenhagen 2024,3,3,FaZe,Natus Vincere,1.460,2.600,9,...,13,2,3.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,2024-04-12,Intl. LAN,IEM Chengdu 2024,3,1,Liquid,FaZe,2.870,1.380,13,...,13,11,10.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN
263,2024-04-12,Intl. LAN,IEM Chengdu 2024,3,3,Virtus.pro,G2,2.250,1.600,6,...,10,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,2024-04-13,Intl. LAN,IEM Chengdu 2024,3,1,Astralis,FaZe,2.210,1.620,10,...,13,5,7.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN
265,2024-04-13,Intl. LAN,IEM Chengdu 2024,3,3,MOUZ,G2,2.170,1.640,13,...,7,13,13.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
cs2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             267 non-null    datetime64[ns]
 1   Type             267 non-null    object        
 2   League           267 non-null    object        
 3   Best Of          267 non-null    int64         
 4   Stars            267 non-null    int64         
 5   Team A           267 non-null    object        
 6   Team B           267 non-null    object        
 7   Moneyline A      267 non-null    float64       
 8   Moneyline B      267 non-null    float64       
 9   R1 Score (A)     267 non-null    int64         
 10  R1 Score (B)     267 non-null    int64         
 11  R2 Score (A)     267 non-null    int64         
 12  R2 Score (B)     267 non-null    int64         
 13  R3 Score (A)     117 non-null    float64       
 14  R3 Score (B)     117 non-null    float64  

In [6]:
cs2['Score Count (A)'] = (
    np.where((cs2['Best Of'] == 3) & (cs2['R1 Score (A)'] > cs2['R1 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 3) & (cs2['R2 Score (A)'] > cs2['R2 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 3) & (cs2['R3 Score (A)'] != 0) & (cs2['R3 Score (A)'] > cs2['R3 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R1 Score (A)'] > cs2['R1 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R2 Score (A)'] > cs2['R2 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R3 Score (A)'] > cs2['R3 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R4 Score (A)'] > cs2['R4 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R5 Score (A)'] != 0) & (cs2['R5 Score (A)'] > cs2['R5 Score (B)']), 1, 0)
)

In [7]:
cs2['Score Count (B)'] = (
    np.where((cs2['Best Of'] == 3) & (cs2['R1 Score (B)'] > cs2['R1 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 3) & (cs2['R2 Score (B)'] > cs2['R2 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 3) & (cs2['R3 Score (A)'] != 0) & (cs2['R3 Score (B)'] > cs2['R3 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R1 Score (B)'] > cs2['R1 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R2 Score (B)'] > cs2['R2 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R3 Score (B)'] > cs2['R3 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R4 Score (B)'] > cs2['R4 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R5 Score (A)'] != 0) & (cs2['R5 Score (B)'] > cs2['R5 Score (A)']), 1, 0)
)


In [8]:
cs2

,Date,Type,League,Best Of,Stars,Team A,Team B,Moneyline A,Moneyline B,R1 Score (A),...,R2 Score (A),R2 Score (B),R3 Score (A),R3 Score (B),R4 Score (A),R4 Score (B),R5 Score (A),R5 Score (B),Score Count (A),Score Count (B)
0,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,1,Aurora,Apeks,1.645,2.200,16,...,13,7,NaN,NaN,NaN,NaN,NaN,NaN,2,0
1,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Metizport,B8,1.833,1.909,13,...,13,8,NaN,NaN,NaN,NaN,NaN,NaN,2,0
2,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Monte,GamerLegion,1.741,2.050,12,...,13,11,13.0,10.0,NaN,NaN,NaN,NaN,2,1
3,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Ninjas in Pyjamas,BlesseD,1.364,3.000,13,...,13,9,NaN,NaN,NaN,NaN,NaN,NaN,2,0
4,2024-03-31,Major,PGL CS2 Major Copenhagen 2024,3,3,FaZe,Natus Vincere,1.460,2.600,9,...,13,2,3.0,13.0,NaN,NaN,NaN,NaN,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,2024-04-12,Intl. LAN,IEM Chengdu 2024,3,1,Liquid,FaZe,2.870,1.380,13,...,13,11,10.0,13.0,NaN,NaN,NaN,NaN,1,2
263,2024-04-12,Intl. LAN,IEM Chengdu 2024,3,3,Virtus.pro,G2,2.250,1.600,6,...,10,13,NaN,NaN,NaN,NaN,NaN,NaN,0,2
264,2024-04-13,Intl. LAN,IEM Chengdu 2024,3,1,Astralis,FaZe,2.210,1.620,10,...,13,5,7.0,13.0,NaN,NaN,NaN,NaN,1,2
265,2024-04-13,Intl. LAN,IEM Chengdu 2024,3,3,MOUZ,G2,2.170,1.640,13,...,7,13,13.0,10.0,NaN,NaN,NaN,NaN,2,1


In [9]:
cs2['Number of Rounds'] = cs2['Score Count (A)'] + cs2['Score Count (B)']

In [10]:
# Initialize the Moneyline Result column
cs2['Moneyline Result'] = np.where(cs2['Score Count (A)'] > cs2['Score Count (B)'],
                                   cs2['Moneyline A'],
                                   np.where(cs2['Score Count (B)'] > cs2['Score Count (A)'],
                                            cs2['Moneyline B'],
                                            np.nan))

In [11]:
conditions = [
    (cs2['Moneyline Result'] == cs2['Moneyline A']) & (cs2['Moneyline A'] < cs2['Moneyline B']),
    (cs2['Moneyline Result'] == cs2['Moneyline B']) & (cs2['Moneyline B'] < cs2['Moneyline A']),
    (cs2['Moneyline Result'] == cs2['Moneyline B']) & (cs2['Moneyline B'] > cs2['Moneyline A']),
    (cs2['Moneyline Result'] == cs2['Moneyline A']) & (cs2['Moneyline A'] > cs2['Moneyline B'])
]

# Results
results = ['Favorite', 'Favorite', 'Underdog', 'Underdog']

# Add Winner column based on Moneyline comparison
cs2['Winner'] = np.select(conditions, results, default=np.nan)

In [12]:
cs2['Winning Team'] = np.where(
    cs2['Moneyline Result'] == cs2['Moneyline A'],
    cs2['Team A'],
    np.where(
        cs2['Moneyline Result'] == cs2['Moneyline B'],
        cs2['Team B'],
        np.nan
    )
)

In [13]:
cs2 = cs2[cs2['Best Of'] == 3]

In [14]:
columns_to_drop = ['R4 Score (A)', 'R4 Score (B)', 'R5 Score (A)', 'R5 Score (B)']

cs2 = cs2.drop(columns=columns_to_drop, axis=1)

In [15]:
cs2.head()

,Date,Type,League,Best Of,Stars,Team A,Team B,Moneyline A,Moneyline B,R1 Score (A),...,R2 Score (A),R2 Score (B),R3 Score (A),R3 Score (B),Score Count (A),Score Count (B),Number of Rounds,Moneyline Result,Winner,Winning Team
0,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,1,Aurora,Apeks,1.645,2.200,16,...,13,7,NaN,NaN,2,0,2,1.645,Favorite,Aurora
1,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Metizport,B8,1.833,1.909,13,...,13,8,NaN,NaN,2,0,2,1.833,Favorite,Metizport
2,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Monte,GamerLegion,1.741,2.050,12,...,13,11,13.0,10.0,2,1,3,1.741,Favorite,Monte
3,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Ninjas in Pyjamas,BlesseD,1.364,3.000,13,...,13,9,NaN,NaN,2,0,2,1.364,Favorite,Ninjas in Pyjamas
4,2024-03-31,Major,PGL CS2 Major Copenhagen 2024,3,3,FaZe,Natus Vincere,1.460,2.600,9,...,13,2,3.0,13.0,1,2,3,2.600,Underdog,Natus Vincere


### Round Winners

In [16]:
cs2 = calculate_round_winner(cs2, 3)

In [17]:
cs2['Underdog Comeback'] = np.where(
    (cs2['Round 2 Winner'] == 'Underdog') & (cs2['Round 3 Winner'] == 'Underdog'),
    'Yes',
    'No'
)


In [18]:
cs2.head()

,Date,Type,League,Best Of,Stars,Team A,Team B,Moneyline A,Moneyline B,R1 Score (A),...,Score Count (A),Score Count (B),Number of Rounds,Moneyline Result,Winner,Winning Team,Round 1 Winner,Round 2 Winner,Round 3 Winner,Underdog Comeback
0,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,1,Aurora,Apeks,1.645,2.200,16,...,2,0,2,1.645,Favorite,Aurora,Favorite,Favorite,,No
1,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Metizport,B8,1.833,1.909,13,...,2,0,2,1.833,Favorite,Metizport,Favorite,Favorite,,No
2,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Monte,GamerLegion,1.741,2.050,12,...,2,1,3,1.741,Favorite,Monte,Underdog,Favorite,Favorite,No
3,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Ninjas in Pyjamas,BlesseD,1.364,3.000,13,...,2,0,2,1.364,Favorite,Ninjas in Pyjamas,Favorite,Favorite,,No
4,2024-03-31,Major,PGL CS2 Major Copenhagen 2024,3,3,FaZe,Natus Vincere,1.460,2.600,9,...,1,2,3,2.600,Underdog,Natus Vincere,Underdog,Favorite,Underdog,No


# Analysis

In [19]:
cs2 = cs2[cs2['Number of Rounds'] == 3]
cs2

,Date,Type,League,Best Of,Stars,Team A,Team B,Moneyline A,Moneyline B,R1 Score (A),...,Score Count (A),Score Count (B),Number of Rounds,Moneyline Result,Winner,Winning Team,Round 1 Winner,Round 2 Winner,Round 3 Winner,Underdog Comeback
2,2024-04-15,Online,ESL Challenger Jonkoping 2024 Europe Closed Qu...,3,0,Monte,GamerLegion,1.741,2.05,12,...,2,1,3,1.741,Favorite,Monte,Underdog,Favorite,Favorite,No
4,2024-03-31,Major,PGL CS2 Major Copenhagen 2024,3,3,FaZe,Natus Vincere,1.460,2.60,9,...,1,2,3,2.600,Underdog,Natus Vincere,Underdog,Favorite,Underdog,No
5,2024-03-30,Major,PGL CS2 Major Copenhagen 2024,3,3,Natus Vincere,G2,2.150,1.64,16,...,2,1,3,2.150,Underdog,Natus Vincere,Underdog,Favorite,Underdog,No
6,2024-03-30,Major,PGL CS2 Major Copenhagen 2024,3,5,Vitality,FaZe,1.710,2.06,7,...,1,2,3,2.060,Underdog,FaZe,Underdog,Favorite,Underdog,No
9,2024-03-28,Major,PGL CS2 Major Copenhagen 2024,3,5,Spirit,FaZe,1.360,3.00,7,...,1,2,3,3.000,Underdog,FaZe,Underdog,Favorite,Underdog,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,2024-04-09,Intl. LAN,IEM Chengdu 2024,3,2,HEROIC,FURIA,1.720,2.04,6,...,2,1,3,1.720,Favorite,HEROIC,Underdog,Favorite,Favorite,No
254,2024-04-09,Intl. LAN,IEM Chengdu 2024,3,1,FlyQuest,Virtus.pro,5.990,1.11,14,...,1,2,3,1.110,Favorite,Virtus.pro,Favorite,Underdog,Favorite,No
262,2024-04-12,Intl. LAN,IEM Chengdu 2024,3,1,Liquid,FaZe,2.870,1.38,13,...,1,2,3,1.380,Favorite,FaZe,Favorite,Underdog,Favorite,No
264,2024-04-13,Intl. LAN,IEM Chengdu 2024,3,1,Astralis,FaZe,2.210,1.62,10,...,1,2,3,1.620,Favorite,FaZe,Favorite,Underdog,Favorite,No


In [20]:
# Round 3 Underdog Winner
r3_ud = cs2['Round 3 Winner'].eq("Underdog").sum()
r3_ud

46

In [21]:
# Round 3 Underdog Winner divided by Total 3 Round Matches
r3_ud_win = r3_ud / cs2['Round 3 Winner'].count()

In [22]:
r3_ud_win.round(5)

0.40708

In [23]:
underdog_comeback = ((cs2['Round 2 Winner'] == "Underdog") & (cs2['Round 3 Winner'] == "Underdog")).sum()
underdog_comeback

21

In [24]:
favorite_comeback = ((cs2['Round 2 Winner'] == "Favorite") & (cs2['Round 3 Winner'] == "Favorite")).sum()
favorite_comeback

30

In [25]:
ud_cb = underdog_comeback / cs2['Round 3 Winner'].count()
ud_cb

0.18584070796460178

In [26]:
fv_cb = favorite_comeback / cs2['Round 3 Winner'].count()
fv_cb

0.26548672566371684

# Results

In [27]:
print("Underdogs win", round(r3_ud_win * 100, 5), "% of the time in round 3.")
print("Fair price =", round(1/r3_ud_win, 5),".")

print()
print("Underdogs comeback at", round(ud_cb * 100, 5), "% of the time.")
print("Fair price =", round(1/ud_cb, 5),".")

Underdogs win 40.70796 % of the time in round 3.
Fair price = 2.45652 .

Underdogs comeback at 18.58407 % of the time.
Fair price = 5.38095 .


In [28]:
print("Favorites win", round((1 - r3_ud_win) * 100, 5), "% of the time in round 3.")
print("Fair price =", round(1/(1 - r3_ud_win), 5),".")

print()
print("Favorites comeback at", round(fv_cb * 100, 5), "% of the time.")
print("Fair price =", round(1/fv_cb, 5),".")

Favorites win 59.29204 % of the time in round 3.
Fair price = 1.68657 .

Favorites comeback at 26.54867 % of the time.
Fair price = 3.76667 .
